- here is my task work within 2 hours.

## Importe Libraries

In [4]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


-  Get the data from link = "https://www.gutenberg.org/cache/epub/100/pg100-images.html"

In [3]:

response = requests.get("https://www.gutenberg.org/cache/epub/100/pg100-images.html")
soup = BeautifulSoup(response.text,"html.parser")


## text presrocessing

In [17]:
text=soup.get_text()
text = text.replace("\r", " ").replace("\n", " ").replace("\t", " ")
text=text.lower()
text = re.sub(r'[^a-z\s]', '', text)

In [18]:
# for sentence in text.split('\n'):
#     print(sentence)

- Stop words are not removed to preserve grammatical structure for text generation and also stemming and lemmatization is not done.


In [ ]:
# Convert text to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

token_list= tokenizer.texts_to_sequences([text])[0]
total_words = len(tokenizer.word_index) + 1 
# total_words= 15000

In [ ]:
# Create input sequences using list of tokens
sequence_length = 20
input_sequences = []

for i in range(sequence_length, len(token_list)):
    input_sequences.append(token_list[i-sequence_length:i+1])

input_sequences = np.array(input_sequences)


In [21]:
input_sequences.shape

(965962, 21)

In [ ]:
# Split data into inputs and targets
X = input_sequences[:, :-1]   
y = input_sequences[:, -1]    

In [23]:
X.shape, y.shape

((965962, 20), (965962,))

In [24]:
# y = to_categorical(y, num_classes=total_words)

In [33]:
x_train=X[:60000]
y_train=y[:60000]
x_test=X[60000:]
y_test=y[60000:]

In [26]:
x_train.shape, y_train.shape

((60000, 20), (60000,))

## Model Building

In [30]:
model = Sequential()
model.add(Embedding(total_words,50, input_length=sequence_length))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

In [31]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Model Training

In [32]:
# early_stop = EarlyStopping(monitor="val_loss",patience=3,restore_best_weights=True)
# , callbacks=[early_stop]
# history = 
model.fit(x_train, y_train, epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 47s 123ms/step - accuracy: 0.0262 - loss: 7.9074 - val_accuracy: 0.0305 - val_loss: 7.0776
Epoch 2/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 42s 112ms/step - accuracy: 0.0322 - loss: 6.6145 - val_accuracy: 0.0348 - val_loss: 7.1144
Epoch 3/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 41s 110ms/step - accuracy: 0.0374 - loss: 6.4762 - val_accuracy: 0.0394 - val_loss: 7.0669
Epoch 4/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 40s 108ms/step - accuracy: 0.0458 - loss: 6.3438 - val_accuracy: 0.0423 - val_loss: 7.0812
Epoch 5/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 42s 111ms/step - accuracy: 0.0536 - loss: 6.2308 - val_accuracy: 0.0468 - val_loss: 7.0868
Epoch 6/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 41s 109ms/step - accuracy: 0.0602 - loss: 6.1322 - val_accuracy: 0.0514 - val_loss: 7.1035
Epoch 7/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 41s 109ms/step - accuracy: 0.0653 - loss: 6.0038 - val_accuracy: 0.0527 - val_loss: 7.1314
Epoch 8/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 43s 116ms/step - accuracy: 0.0707 - loss: 5

In [34]:
model.predict(x_test)

 9578/28312 ━━━━━━━━━━━━━━━━━━━━ 3:45 12ms/step

: 